In [1]:
import pokebase as pb 

In [2]:
import pandas as pd

df = pd.read_csv('hgss-moves.csv')

In [3]:
df.head()

,TM,Move,Effect,Type,Kind,Power,Accuracy,PP,Location
0,TM01,Focus Punch,A powerful loyalty attack. The user flinches i...,Fighting,Physical,150,100,20,Cianwood City Gym
1,TM02,Dragon Claw,Slashes the foe with sharp claws.,Dragon,Physical,80,100,15,Route 27 Goldenrod City Lottery (Sunday)
2,TM03,Water Pulse,Attacks with ultrasonic waves. May confuse the...,Water,Special,60,100,20,Cerulean City Gym
3,TM04,Calm Mind,Raises SP. ATK and SP. DEF by focusing the mind.,Psychic,Other,--,--,20,Battle Frontier Shop (48 B.P.)
4,TM05,Roar,Makes the foe flee to end the battle.,Normal,Other,--,100,20,Route 32


In [4]:
def row_to_text(row):
    return (
        f"{row['Move']} is a {row['Type']} type {row['Kind']} move with {row['Power']} power, "
        f"{row['Accuracy']} accuracy, and {row['PP']} PP. "
        f"It is described as: {row['Effect']}. "
        f"It can be found at: {row['Location']}."
    )

sentences = df.apply(row_to_text, axis=1).tolist()

In [5]:
from sentence_transformers import SentenceTransformer
import faiss 
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(sentences, show_progress_bar = True)
embeddings = np.array(embeddings)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

c:\Users\ali49\Documents\hgss-llm\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 4/4 [00:00<00:00,  4.23it/s]


In [6]:
def semantic_search(query, k=3):
    query_vec = model.encode([query])
    D, I = index.search(np.array(query_vec), k)
    return [sentences[i] for i in I[0]]


In [7]:
from huggingface_hub import login

login('PLACE_HOLDER2')

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_path = "C:/Users/ali49/Documents/hgss-llm/model"

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)

qa = pipeline("text-generation", model=model, tokenizer=tokenizer, device=-1)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


OSError: The paging file is too small for this operation to complete. (os error 1455)

In [ ]:
response = qa("What type is Focus Punch?", max_new_tokens=50, do_sample=False)
print(response[0]["generated_text"])